In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd


*Python script to extract league standing*

In [ ]:


# List of URLs to scrape
urls = [
    {"url": "https://www.skysports.com/ligue-1-table", "source": "Ligue 1"},
    {"url": "https://www.skysports.com/premier-league-table", "source": "Premier League"},
    {"url": "https://www.skysports.com/la-liga-table", "source": "la liga"},
    {"url": "https://www.skysports.com/bundesliga-table", "source": "Bundesliga"},
    {"url": "https://www.skysports.com/serie-a-table", "source": "Seria A"},
    {"url": "https://www.skysports.com/eredivisie-table", "source": "Eredivisie"},
    {"url": "https://www.skysports.com/scottish-premier-table", "source": "Scottish premiership"}

    # Add more URLs as needed
]

# Create an empty list to store DataFrames
dfs = []

for url_info in urls:
    url = url_info["url"]
    source = url_info["source"]

    # Send HTTP Request and Parse HTML
    r = requests.get(url)
    soup = BeautifulSoup(r.text, "lxml")

    # Find and Extract Table Headers
    table = soup.find("table", class_="standing-table__table")
    headers = table.find_all("th")
    titles = [i.text for i in headers]

    # Create an Empty DataFrame
    df = pd.DataFrame(columns=titles)

    # Iterate Through Table Rows and Extract Data
    rows = table.find_all("tr")
    for i in rows[1:]:
        data = i.find_all("td")
        row = [tr.text.strip() for tr in data]  # Apply .strip() to remove \n
        l = len(df)
        df.loc[l] = row

    # Add a column for source URL
    df["Source"] = source

    # Append the DataFrame to the list
    dfs.append(df)

# Concatenate all DataFrames into a single DataFrame
football_standing = pd.concat(dfs, ignore_index=True)

#remove first teo column

football_standing = football_standing.iloc[:, 1:]

# Print the merged DataFrame
print(football_standing)


*Python script to extract scores from game played*

In [ ]:


url = "https://www.skysports.com/football-results"
r = requests.get(url)
soup = BeautifulSoup(r.text, "lxml")

home_team = soup.find_all("span", class_="matches__item-col matches__participant matches__participant--side1")
x = [name.strip() for i in home_team for name in i.stripped_strings]

scores = soup.find_all("span", class_="matches__teamscores")
s = [name.strip().replace('\n\n', '\n') for i in scores for name in i.stripped_strings]
appended_scores = [f"{s[i]}\n{s[i+1]}".replace('\n', ' ') for i in range(0, len(s), 2)]

away_team = soup.find_all("span", class_="matches__item-col matches__participant matches__participant--side2")
y = [name.strip() for i in away_team for name in i.stripped_strings]

# Make sure all arrays have the same length
min_length = min(len(x), len(appended_scores), len(y))
data = {"Home Team": x[:min_length], "Scores": appended_scores[:min_length], "Away Team": y[:min_length]}

# Create a DataFrame
footballscores = pd.DataFrame(data)
print(footballscores)


*To connect with your AWS s3 bucket, you will be required to input keys upon running the below code*

In [ ]:
!pip install awscli
!aws configure

import boto3
from io import StringIO

In [ ]:
# Concatenate all DataFrames into a single DataFrame
footballstanding = pd.concat(dfs, ignore_index=True)
footballstanding = footballstanding.iloc[:, 1:]

# convert DataFrame to CSV
csv_buffer = StringIO()
footballstanding.to_csv(csv_buffer)

s3_resources = boto3.resource('s3')

# footballstanding was erroneously used as my bucket name. Please use another unique name
s3_resources.Object('footballstanding','footballstanding.csv').put(Body=csv_buffer.getvalue())

In [ ]:
# convert DataFrame to CSV
csv_buffer = StringIO()
footballscores.to_csv(csv_buffer)

s3_resources = boto3.resource('s3')

s3_resources.Object('footballstanding','footballscores.csv').put(Body=csv_buffer.getvalue())